# Experiment with parallel segmentation 

_The main idea of this experints is to check how the accuracy is affected if use a bunch of binray classifiers instead of one multiclass classifier. 
And then combine it together to get a multiplexed result._

1. Load dataset

In [1]:
import os
import sys

import keras.src.backend

sys.path.append("/mnt/d/aspiranture/code/sat_image_segmentation")
import dataset_tool.dataset6_generator

2024-12-03 20:46:10.192666: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 20:46:11.367941: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import importlib

In [3]:
import dataset_6_binary_tree
importlib.reload(dataset_6_binary_tree)
import dataset_tool.binary_helper
importlib.reload(dataset_tool.binary_helper)

<module 'dataset_tool.binary_helper' from '/mnt/d/aspiranture/code/sat_image_segmentation/dataset_tool/binary_helper.py'>

In [4]:
dataset = dataset_tool.dataset6_generator.get_dataset6()

Error read []
len 27


In [5]:
categories = list(dataset.keys())

In [6]:
categories

['water', 'forest', 'fields', 'city', 'sands', 'mountains']

In [7]:
import numpy as np

In [8]:
x_data = []
y_data = []
for idx, category in enumerate(categories):
    x_data.append(dataset[category])
    y = np.full(dataset[category].shape[1], idx)
    y_data.extend(y)

In [9]:
x_data_1 = np.hstack(x_data)

In [10]:
x_data_1.shape

(12, 11581761)

In [11]:
y_data_1 = np.array(y_data)

In [12]:
y_data_1 = y_data_1 + 1

In [13]:
y_data_1.shape

(11581761,)

In [14]:
x_data_1 = np.moveaxis(x_data_1, 0, -1)

In [15]:
x_data_1.shape

(11581761, 12)

In [16]:
categories_map = {category: idx+1 for idx, category in enumerate(categories)}

In [17]:
categories_map

{'water': 1, 'forest': 2, 'fields': 3, 'city': 4, 'sands': 5, 'mountains': 6}

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x_data_1, y_data_1, test_size=0.2)

In [38]:
## This code creates N-1 binary classifiers for each category

classifiers = {}
classifiers_test = {}

for key, value in categories_map.items():
    print(value)
    y_train_1 = y_train.copy()
    y_train_1[y_train_1 != value] = 0
    y_train_1[y_train_1 == value] = 1
    y_test_1 = y_test.copy()
    y_test_1[y_test_1 != value] = 0
    y_test_1[y_test_1 == value] = 1
    classifiers[key] = y_train_1
    classifiers_test[key] = y_test_1

1
2
3
4
5
6


In [21]:
y_train

array([1, 1, 3, ..., 1, 3, 1])

In [22]:
classifiers['water']

array([1, 1, 0, ..., 1, 0, 1])

## Create binary classifier to each category using splittend and formatted train data
____

This aproachpotenitally gives more accurate resut, bacuse we dont't decrease dataset for each iteration and give a model
opertunity to learn only for specific category. This methodology decrease risks for false positive and false negative results.

In [23]:
import tensorflow as tf
import layers
from tensorflow.keras.layers import *

In [24]:
tf.__version__

'2.16.1'

In [25]:
from tensorflow.keras.backend import *

In [26]:
def create_model():
    """
    Creates binary model keras instance
    :return:
    """
    model = tf.keras.models.Sequential([
        layers.naive_bayes.MultiClassNaiveBayesLayer(num_features=12, num_of_categories=2, input_shape=(12,)),
        Activation("softmax")
    ])
    
    # Assuming you've trained or set the weights, you can then save the model:
    # save_model(model, 'naive_bayes_model.h5')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'mse'])
    #model.summary()
    return model

In [27]:
        # means = []
        # variances = []
        # 
        # num_classes = 2  # or however many you have
        # 
        # for i in range(num_classes):
        #     means.append(np.mean(x_data[y_data == i], axis=0))
        #     variances.append(np.var(x_data[y_data == i], axis=0))
        # 
        # means = np.array(means)
        # variances = np.array(variances)
        # return means, variances

In [28]:
models = {}

for key, value in classifiers.items():
    print(key)
    model = create_model()
    
    num_classes = 2
    
    means = []
    variances = []
    
    for i in range(num_classes):
        means.append(np.mean(x_train[value == i], axis=0))
        variances.append(np.var(x_train[value == i], axis=0))
    
    means = np.array(means)
    variances = np.array(variances)
    
    model.layers[0].set_weights([means, variances])
    models[key] = model
    # model.fit(x_train, value, epochs=100, batch_size=32)
    # models[key] = model

water


/mnt/d/aspiranture/code/sat_image_segmentation/layers/naive_bayes.py:43: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(MultiClassNaiveBayesLayer, self).__init__(**kwargs)
2024-12-03 20:48:24.214458: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-03 20:48:24.290108: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-03 20:48:24.290171: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built withou

forest
fields
city
sands
mountains


In [39]:
pred1 = models['water'].predict(x_test, batch_size=2048)

1132/1132 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [35]:
from sklearn.metrics import accuracy_score, classification_report

In [40]:
pred1_1 = pred1.argmax(axis=-1)

In [41]:
accuracy_score(classifiers_test['water'], pred1_1)

0.9604015450149438

In [28]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2023-12-16 13:56:42.234899: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-16 13:56:42.235080: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-16 13:56:42.235127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-16 13:56:42.235474: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-16 13:56:42.235488: I tensorflow/core/co

True

da_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-16 13:56:42.235557: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /device:GPU:0 with 21784 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


In [29]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [30]:
from sklearn.metrics import accuracy_score

In [31]:
pred_water = pred1.argmax(axis=-1)

NameError: name 'pred1' is not defined

In [32]:
accuracy_score(classifiers['water'], pred_water)

NameError: name 'pred_water' is not defined

In [42]:
predictions_1 = {}
for key, value in models.items():
    print(key)
    pred = value.predict(x_test, batch_size=2048)
    predictions_1[key] = pred

water
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
forest
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
fields
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
city
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
sands
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
mountains
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [30]:
import pandas as pd

In [31]:
df_1 = pd.DataFrame(predictions_1)

ValueError: Per-column arrays must each be 1-dimensional

In [32]:
predictions_1

{'water': array([[0., 1.],
        [0., 1.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]], dtype=float32),
 'forest': array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [1., 0.],
        [1., 0.]], dtype=float32),
 'fields': array([[1., 0.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 0.],
        [0., 1.],
        [0., 1.]], dtype=float32),
 'city': array([[1., 0.],
        [1., 0.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]], dtype=float32),
 'sands': array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]], dtype=float32),
 'mountains': array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]], dtype=float32)}

In [35]:
categories

['water', 'forest', 'fields', 'city', 'sands', 'mountains']

In [36]:
y_data_1 = np.array(y_data)

In [37]:
xx_data_2 = []

for value in predictions_1.values():
    xx_data_2.append(value)

In [38]:
xx_data_2 = np.asarray(xx_data_2)

In [39]:
xx_data_2.shape

(6, 9265408, 2)

In [40]:
xx_data_2_1 = np.moveaxis(xx_data_2, 0, 1)

In [41]:
xx_data_2_1.shape

(9265408, 6, 2)

In [42]:
xx_data__2 = np.reshape(xx_data_2_1, (xx_data_2_1.shape[0], xx_data_2_1.shape[1] * xx_data_2_1.shape[2]))

In [43]:
xx_data__2.shape

(9265408, 12)

In [126]:
def create_model2():
    """
    Creates binary model keras instance
    :return:
    """
    model = tf.keras.models.Sequential([
        #Dense(128, activation='relu', input_shape=(2,6)),
        Conv1D(128, 2, activation='relu',input_shape=(6,2)),
        Conv1D(64, 2, activation='relu'),
        Flatten(),
        #Dense(128, activation='relu', input_shape=(12,)),
        #Dense(64),
        Dense(32),
        Dense(7),
        #layers.naive_bayes.MultiClassNaiveBayesLayer(num_features=12, num_of_categories=2, input_shape=(12,)),
        Activation("softmax")
    ])
    
    # Assuming you've trained or set the weights, you can then save the model:
    # save_model(model, 'naive_bayes_model.h5')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'mse'])
    #model.summary()
    return model

In [127]:
model_2 = create_model2()

In [128]:
model_2.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 5, 128)            640       
                                                                 
 conv1d_13 (Conv1D)          (None, 4, 64)             16448     
                                                                 
 flatten_7 (Flatten)         (None, 256)               0         
                                                                 
 dense_33 (Dense)            (None, 32)                8224      
                                                                 
 dense_34 (Dense)            (None, 7)                 231       
                                                                 
 activation_20 (Activation)  (None, 7)                 0         
                                                                 
Total params: 25543 (99.78 KB)
Trainable params: 2554

In [44]:
def create_model_multi_bayes():
    """
    Creates binary model keras instance
    :return:
    """
    model = tf.keras.models.Sequential([
        layers.naive_bayes.MultiClassNaiveBayesLayer(num_features=12, num_of_categories=7, input_shape=(12,)),
        Activation("softmax")
    ])
    
    # Assuming you've trained or set the weights, you can then save the model:
    # save_model(model, 'naive_bayes_model.h5')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'mse'])
    #model.summary()
    return model

In [45]:
model_multi_bayes = create_model_multi_bayes()
model_multi_bayes.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_class_naive_bayes_la  (None, 7)                 168       
 yer_6 (MultiClassNaiveBaye                                      
 sLayer)                                                         
                                                                 
 activation_6 (Activation)   (None, 7)                 0         
                                                                 
Total params: 168 (672.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 168 (672.00 Byte)
_________________________________________________________________


In [46]:
y_train_categorical = tf.keras.utils.to_categorical(y_train)

In [47]:
y_train_1 = y_train.copy()

In [65]:
num_classes_multi = 7

means_multi = []
variances_multi = []

for i in range(num_classes_multi):
    means_multi.append(np.mean(xx_data__2[y_train_1 == i], axis=0))
    variances_multi.append(np.var(xx_data__2[y_train_1 == i], axis=0))

means_multi = np.array(means_multi)
variances_multi = np.array(variances_multi)

means_multi = np.nan_to_num(means_multi)
variances_multi = np.nan_to_num(variances_multi)

model_multi_bayes.layers[0].set_weights([means_multi, variances_multi])
model_multi_bayes.layers[0].trainable = True


# models[key] = model

/home/andrew/miniconda3/envs/segmentation-new/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/andrew/miniconda3/envs/segmentation-new/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3747: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [49]:
model_multi_bayes.fit(xx_data__2, y_train_categorical, epochs=100, batch_size=2048)

Epoch 1/100
4525/4525 [==============================] - 41s 9ms/step - loss: 2356392230912.0000 - accuracy: 0.8251 - mse: 0.0439
Epoch 2/100
4525/4525 [==============================] - 41s 9ms/step - loss: 2356397473792.0000 - accuracy: 0.8251 - mse: 0.0439
Epoch 3/100
4525/4525 [==============================] - 40s 9ms/step - loss: 2356397211648.0000 - accuracy: 0.8251 - mse: 0.0439
Epoch 4/100
4525/4525 [==============================] - 40s 9ms/step - loss: 2356395900928.0000 - accuracy: 0.8251 - mse: 0.0439
Epoch 5/100
4525/4525 [==============================] - 40s 9ms/step - loss: 2356397211648.0000 - accuracy: 0.8251 - mse: 0.0439
Epoch 6/100
4525/4525 [==============================] - 40s 9ms/step - loss: 2356399570944.0000 - accuracy: 0.8251 - mse: 0.0439
Epoch 7/100
4525/4525 [==============================] - 41s 9ms/step - loss: 2356391444480.0000 - accuracy: 0.8251 - mse: 0.0439
Epoch 8/100
3429/4525 [=====================>........] - ETA: 9s - loss: 2357067513856.000

KeyboardInterrupt: 

In [134]:
#model_2.fit(xx_data_2_1, y_train_categorical, epochs=100, batch_size=2048)

Epoch 1/100


2023-12-02 00:44:10.659077: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


4525/4525 [==============================] - 66s 14ms/step - loss: 0.2542 - accuracy: 0.9191 - mse: 0.0178
Epoch 2/100
4525/4525 [==============================] - 46s 10ms/step - loss: 0.2451 - accuracy: 0.9215 - mse: 0.0173
Epoch 3/100
4525/4525 [==============================] - 56s 12ms/step - loss: 0.2443 - accuracy: 0.9217 - mse: 0.0173
Epoch 4/100
 122/4525 [..............................] - ETA: 1:07 - loss: 0.2455 - accuracy: 0.9211 - mse: 0.0174

KeyboardInterrupt: 

In [81]:
model_2.save('model_2.keras')

#### Test with validation data 

In [50]:
validate_predictions_1 = {}

for key, value in models.items():
    # print(key)
    pred = value.predict(x_test, batch_size=1024)
    validate_predictions_1[key] = pred

2263/2263 [==============================] - 8s 4ms/step


In [51]:
xx_data_validate_2 = []

for value in validate_predictions_1.values():
    xx_data_validate_2.append(value)

In [52]:
xx_data_validate_2 = np.asarray(xx_data_validate_2)

In [53]:
xx_data_validate_2.shape

(6, 2316353, 2)

In [54]:
xx_data_validate_2_1 = np.moveaxis(xx_data_validate_2, 0, 1)

In [55]:
xx_data_validate_2_1.shape

(2316353, 6, 2)

In [56]:
xx_data_validate__2 = np.reshape(xx_data_validate_2_1, (xx_data_validate_2_1.shape[0], xx_data_validate_2_1.shape[1] * xx_data_validate_2_1.shape[2]))

In [66]:
xx_data_validate__2_preds = model_multi_bayes.predict(xx_data_validate__2, batch_size=1024)

2263/2263 [==============================] - 13s 6ms/step


In [67]:
#xx_data_validate__2_preds = model_2.predict(xx_data_validate_2_1, batch_size=1024)

In [68]:
xx_data_val = xx_data_validate__2_preds.argmax(axis=-1)

In [69]:
xx_data_val.shape

(2316353,)

In [70]:
from sklearn.metrics import accuracy_score

In [71]:
y_test.max()

6

In [72]:
xx_data_val.max()

6

In [73]:
accuracy_score(y_test, xx_data_val)

0.8250275325047607

## Check without neural network using max value for probability

In [74]:
probs_1 = xx_data_validate_2[:,:,1]

In [75]:
probs_1 = np.moveaxis(probs_1, 0, 1)

In [76]:
probs_1.shape

(2316353, 6)

In [77]:
probs_1_cat = np.argmax(probs_1, axis=-1)

In [78]:
probs_1_cat = probs_1_cat + 1

In [79]:
accuracy_score(y_test, probs_1_cat)

0.8883680509835936

## Validate with test image